Pre-Procesing of age recommendation files. 

In [2]:
from google.colab import drive 
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
import pandas as pd
cols_1 = ["Title","Reading_age", "Desc"]
df_1 = pd.read_csv("/gdrive/My Drive/MSC_Thesis_Project/Data/children_books.csv", usecols= cols_1)

# Get reading age column
ages_1 = df_1['Reading_age']

# Reading age column can contain strings of the format AGE or AGE+ or AGE-AGE.
# Code below modifies AGE, AGE+ strings based on numerical value of AGE.
## If AGE < 10, modify string to "AGE-10", i.e. upper age limit is 10
## If 10 <= AGE < 18, modify string to "AGE-18", i.e. upper age limit is 18
age=""

for index, value in ages_1.items():
   if value.rstrip().endswith("+") :
       age = value.rstrip().replace("+","")
       if int(age) < 10 :
          value = age + "-10"
       else :
          value = age + "-18"
   elif value.rstrip().isnumeric() :    
       age = value.rstrip()  
       if int(age) < 10 :
          value = age + "-10"
       else :
          value = age + "-18"
   else : 
       value = value.rstrip()       
   ages_1[index]= value   

# ages_1 now contains strings of the form "LOWER-UPPER" age
#display df_1
df_1

,Title,Desc,Reading_age
0,The Girl Who Drank the Moon,"Every year, the evil Protectorate offers a bab...",10-14
1,Time Between Us,Sixteen-year-old Anna is struggling to underst...,12-18
2,Girl Out of Water,Lou Brown's swimming ambitions sank without tr...,10-18
3,Captive,Robyn is scared. Ever since the attempted assa...,13-18
4,The School of Music,Welcome to the School of Music. In charge is M...,10-18
...,...,...,...
3264,Spy for the Queen of Scots,"Beautiful young aristocrat Ginette, known as J...",12-18
3265,Jimmy Corrigan:,This extraordinary graphic novel is so obvious...,14-18
3266,The Martian Girl,"Following on from the brilliant Lost on Mars, ...",10-18
3267,Buffalo Soldier,Plantation slave Charley Smith is eleven when ...,11-18


In [4]:
cols_2 = ["names","cats","desc"]
df_2 = pd.read_csv("/gdrive/My Drive/MSC_Thesis_Project/Data/children_stories_UTF8.csv", usecols= cols_2)
ages_2 = df_2['cats']
age=""

#ages_2 has categories data as - eg. 'Age 3-6'
#split the data into two strings 'Age' and '3-6'
#retain the second value '3-6' and discard 'Age' string
#some rows have the value as only 'Age', replace the string with '0-0'
for i in ages_2.index:
    if len(ages_2[i].split())>=2 :
       age= ages_2[i].split()[1]
    else :
       age = "0-0" 
       
    ages_2[i] = age

#some rows in ages_2 have the values as '11+', '3+', etc.,
#such values should be modified into '11-18', '3-10'
#if lower age is less than 7, the upper age should be defined as 7 
#if lower age is less than 10, the upper age should be defined as 10 
#if lower age is greater than 10, the upper age ahould be defined as 18
#some rows have values as '6months+' such rows should be modified as '6m-5'

for i in ages_2.index:
    age=""
    if(ages_2[i].endswith("+")):
      age = ages_2[i].replace("+","")
      if(age.isnumeric()):
         if(int(age)<7):
            age = age + "-7"
         elif(int(age)<10):   
            age = age + "-10"
         elif(int(age)<14):
            age = age + "-14" 
         elif(int(age)<18):
            age = age + "-18"   
      else:
         age = age+"-5"   
      ages_2[i] = age


#swapping the 'cats' and 'desc' columns in df_2 
#so that we can concatenate df_1 & df_2 to form df_3 in the upcoming code  
#swapping columns 
cols  = list(df_2.columns)      
a, b = cols.index('cats'), cols.index('desc')
cols[b], cols[a] = cols[a], cols[b]
df_2 = df_2[cols]

#display df_2
df_2

,names,desc,cats
0,HIDE AND SEEK,Was it just another game of hide and seek? No....,2-9
1,GINGER THE GIRAFFE,Read this warm tale of camaraderie and affecti...,2-9
2,DOING MY CHORES,Love shines through this great illustrated kid...,2-9
3,ABE THE SERVICE DOG,Abe was a real Service Dog who dedicated his l...,2-9
4,SUNNY MEADOWS WOODLAND SCHOOL,The class took a little train and went deep in...,2-9
...,...,...,...
425,Carrying the Elephant: A Memoir of Love and Loss,In the 72 prose poems that make up this unusua...,11-14
426,War and Peas,Nearly forty years after its original appearan...,8-10
427,Love that Dog,"Jack has a great sadness in his life, but he i...",9-12
428,A Pilgrim's Progress,'I had a dream last night ... large enough to ...,9-10


In [5]:
#renaming the columns of df_2 
#'cats' column renamed as 'Reading_age' 
#'desc' column renamed as 'Desc'
#so it becomes easy to concatenate df_1 & df_2 
df_2.rename(columns={'cats':'Reading_age', 'desc':'Desc', 'names':'Title'}, inplace = True)

#In the code below, 
#we are extracting suggested 'age_range' and 'book_synopsis' 
#from two different dataframes df_1 & df_2 into a single dataframe df_3 
#reset_index is needed to maintain the index. Otherwise, the df is joined as 0-n and 0-m format.
df_3 = pd.concat([df_1, df_2]).reset_index(drop=True)

# removing special characters in description 
description = df_3['Desc']
spl_indices= []
encode_str =""
decode_str =""

#identifying the descriptions with special characters
for i in description.index:
   encode_str = description[i].encode("ascii","ignore")
   decode_str = encode_str.decode()
   if (len(description[i]) != len(decode_str)):
        spl_indices.append(i)
   
#using iloc() function to retireve the records with special characters. 
k= len(spl_indices)-1

for x in spl_indices:
    encode_str= description.iloc[x].encode("ascii","ignore")
    description.iloc[x] = encode_str.decode()




In [6]:
#Sorting the rows by book name 
df_3.sort_values('Title', inplace=True)

#Dropping the duplicate rows 
df_4 = df_3.drop_duplicates(subset='Desc', keep= "first")

df_4




,Title,Desc,Reading_age
3282,A TRICK TRIKE,Little Ross was sad. He didnt have a nice new ...,2-9
3272,ABE THE SERVICE DOG,Abe was a real Service Dog who dedicated his l...,2-9
3287,BUBBLE FUN,"When the last bubble popped, he was gone but a...",2-9
3278,CAPTAIN FANTASTIC,"Dr. Zobs evil plans had failed once again, th...",2-9
3281,DO YOU WONDER WHY BUGS ARE GOOD?,Those little beauties have inspired so many st...,2-9
...,...,...,...
2496,Witchstruck,The future Elizabeth I has been imprisoned by ...,12-18
3672,Wolf Brother,"Thousands of years ago, the land is a dark for...",9-10
3455,Za-za's Baby Brother,"When Za-za's mum has a baby, it's hard for Za-...",2-7
3388,Zagazoo,Meet Bella and George. Theyre a happy couple w...,3-5


In [7]:
from numpy import NAN
#For the reason that the data from children_books.csv had largely duplicate values. 
#the size of the combined dataframe df_4 had shrunk from 3699 rows to 434 rows. 
#This data is not sufficient to conduct the machine learning process. 
#Have searched for new dataset and found https://www.kaggle.com/modhiibrahimalmannaa/1000-children-books-on-amazom
c = ['Name','Description','Age']
df_new = pd.read_csv("/gdrive/My Drive/MSC_Thesis_Project/Data/books_data.csv", usecols= c)

#sorting rows in df_new 
df_new.sort_values('Name', inplace = True)

#droping duplicate rows 
df_new = df_new.drop_duplicates(subset ='Name', keep = "first")

#The ages are of the form - 
# 'Ages: 5 - 7 years', 'Ages: 12 years and up', ''Ages: 1 year and up''
# 'Great On Kindle: A high quality digital reading experience.Ages: 8 - 11 years'
# 'Great On Kindle: A high quality digital reading experience.', 'Ages 4-8 nan'
# df_new.Age.unique() is used to find the unique values of 'Age' column

#df_new.loc[df_new['Age']== "Ages: 5 - 7 years"]  --> used to retrieve records from the dataframe for a given column value. 

#identifying the rows where Age value starts from 'Ages:' and ends with 'years'
#strip() right and strip() left to obtain the age range, eg. 5-7
#df_new.loc[type(df_new['Age'])== float]

count = 0
#firstly, identifying the rows which have column 'Age' value as 'nan'
#observe that there are 79 such records which have 'Age' as floating value, This should be changed to string value 
#yet the coulumn 'Name' of the book has the age suggestion. The age suggestion from the book-name should be extracted and inserted into Age column
for i in df_new.index: 
    if ( type(df_new['Age'][i]) == float)  :
       df_new['Age'][i] = " "
#       print(df_new['Name'][i]+"\n"+str(df_new['Age'][i]))    



In [8]:
#Now that the column values of rows have been changed from 'float' to string
#Extract age from the 'Name' column (only for those rows who have the age specified in Name column)
#Attach the extracted age range to the 'Age' column. 
for i in df_new.index:
    if (df_new['Name'][i].find("Age") != -1 and df_new['Age'][i]==" "):
       print(df_new['Name'][i]+"\n"+str(df_new['Age'][i]))    

# The following values are to be extracted from different rows - 
# Ages 4-8
# Age 5-7
# Age 5 to 6
# Ages 3 to 5
# Ages 5 and Up

pos = 0
# retrieving book names which have substirng = 'Ages '
# Outputs - Ages 5-8  Ages 4-8  Ages 1-3) Ages 4-8: 101
# Ages 3-5 (Kin   Ages 5-7: Rea   Ages 3-5: Tra   Ages 3-5 (Pre
# Ages 2-4: Beg   Ages 3 to 5,    Ages 5 and Up   Ages 3-5 Read
# Ages 3-5. ABC   Ages 3-5    Ages 4-8
for i in df_new.index: 
   if (df_new['Name'][i].find("Ages") != -1 and df_new['Age'][i]==" "):
      pos = df_new['Name'][i].find("Ages")
      df_new['Age'][i] = df_new['Name'][i][pos: pos+13]
      #print(df_new['Name'][i][pos: pos+13] )


200 Must Know Sight Words Activity Workbook: Learn, Trace & Practice The 200 Most Common High Frequency Words For Kids Learning To Write & Read. | Ages 5-8
 
Dinosaur Coloring Book for Kids: Great Gift for Boys & Girls, Ages 4-8
 
Don't Push the Button! An Easter Surprise: (Easter Board Book, Interactive Books For Toddlers, Childrens Easter Books Ages 1-3)
 
Dot To Dot Books For Kids Ages 4-8: 101 Fun Connect The Dots Books for Kids Age 3, 4, 5, 6, 7, 8 | Easy Kids Dot To Dot Books Ages 4-6 3-8 3-5 6-8 (Boys & Girls Connect The Dots Activity Books)
 
Kindergarten Math Activity Workbook: For Kindergarten and Preschool Kids Learning The Numbers And Basic Math. Tracing Practice Book. | Ages 3-5 (Kindergarten Math Workbook)
 
Kindergarten Math Workbook: Kindergarten and 1st Grade Workbook Age 5-7 | Homeschool Kindergarteners | Addition and Subtraction Activities + Worksheets (Homeschooling Activity Books)
 
Learn to Read: A Magical Sight Words and Phonics Activity Workbook for Beginning Re

In [9]:
#for i in df_new.index:
#  if (df_new['Name'][i].find("Ages") != -1 ):
#      print(df_new['Age'][i])
pos= 0
#retrieving records with age as blank value and book name having 'Age' in its string
#Output - 
#Age 5-7 | 
#Age 5 to 6
# replacing 'Age' column of the row with value retrieved from Book name 
for i in df_new.index:
  if (df_new['Name'][i].find("Age") != -1 and df_new['Age'][i]==" "):
     pos = df_new['Name'][i].find("Age")
     df_new['Age'][i] = df_new['Name'][i][pos: pos+10]



In [10]:
pos = 0
#retrieving records with 'Age' as " " and the book name containing string ' age' in them. 
# there by updating the 'Age' field appropriately 
for i in df_new.index: 
    if ( df_new['Age'][i] == " " and df_new['Name'][i].find(" age") !=-1)  :
       pos = df_new['Name'][i].find(" age") 
       df_new['Age'][i] = df_new['Name'][i][pos+1: pos+9]
       print(df_new['Age'][i], df_new['Name'][i], str(i))

ages 3-5 Scissor Skills Preschool Workbook for Kids: A Fun Cutting Practice Activity Book for Toddlers and Kids ages 3-5: Scissor Practice for Preschool ... 40 Pages of Fun Animals, Shapes and Patterns 135


In [11]:
#Retrieve records which are of the pattern Age = 'Ages: 5 - 7 years'
#change the value to 5-7, removing 'Ages:' and 'years'
for i in df_new.index:
    if(df_new['Age'][i].startswith("Ages: ") and df_new['Age'][i].endswith(" years") ):
        df_new['Age'][i]= df_new['Age'][i].split()[1]+df_new['Age'][i].split()[2]+df_new['Age'][i].split()[3]



In [12]:
# Retrieve records which are of the pattern Age = "Ages: 12 years and up"
# change the value to by adding the upper limit as 18, eg. 12-18
#if(age.isnumeric()):
 #        if(int(age)<7):
 #          age = age + "-7"
 #      elif(int(age)<10):   
 #           age = age + "-10"
 #        elif(int(age)<14):
 #           age = age + "-14" 
 #        elif(int(age)<18):
 #           age = age + "-18"   
 #     else:
 #        age = age+"-5"   
age = "" 
for i in df_new.index:
    if(df_new['Age'][i].startswith("Ages: ") and df_new['Age'][i].endswith(" years and up")):
         age = df_new['Age'][i].split()[1]      #retrieve lower age limit
         if   (int(age)<5)      :               #define upper age limit 
            age= age+"-5"
         elif (int(age) < 7)    :
            age= age+"-7"
         elif (int(age) < 10)   :
            age= age+"-10"
         elif (int(age) < 14)   :
            age= age+"-14"
         elif (int(age) < 18)   :
            age= age+"-18"
         df_new['Age'] [i] = age 


In [13]:
#df_new['Age'].unique()
#identifying the rows with Age = "Great On Kindle: A high quality digital reading experience.Ages: 8 - 11 years"
#split() positions -              0     1  2       3 4    5       6       7       8                9 10 11 12
#extract age range and update 'Age' column
a=[]

for i in df_new.index: 
    if df_new['Age'][i].startswith("Great On Kindle"):
        a = df_new['Age'][i].split()
        if(len(a)>9):
           age = a[9]+a[10]+a[11]   
           df_new['Age'][i]= age 
        else:
           df_new['Age'][i] = "" 

In [14]:
#Cleaning the records with following data in 'Age' column- 
#Data (length = 8)  Eg. Ages 5-8     
#Data (length = 9)  Eg. Ages 1-3) 
#Data (length = 10) Eg. Age 5-7 |      Age 5 to 6
temp= ""

for i in df_new.index: 
   if(df_new['Age'][i].startswith("Age") and len(df_new['Age'][i])==8):
        temp = df_new['Age'][i].split()[1]
        df_new['Age'][i] = temp
   elif(df_new['Age'][i].startswith("Age") and len(df_new['Age'][i])==9):     
        temp = df_new['Age'][i].split()[1][0:3]
        df_new['Age'][i] = temp
   elif(df_new['Age'][i].startswith("Age") and len(df_new['Age'][i])==10 and df_new['Age'][i].find("to")==-1):
        temp = df_new['Age'][i].split()[1]
        df_new['Age'][i] = temp
   
      

In [15]:
#Data (length = 10) Eg. Age 5 to 6
#Data (length = 13) 
#Eg. Ages 4-8: 101  Ages 3-5 (Kin   Ages 3-5. ABC   Ages 3-5 Read   Ages 3-5 (Pre
#Data (length = 19) Eg. Ages: 1 year and up    

for i in df_new.index: 
    if (df_new['Age'][i].startswith("Age ") and len(df_new['Age'][i])==10 and df_new['Age'][i].find("to")!=-1):
        temp = df_new['Age'][i].split()[1]+"-"+ df_new['Age'][i].split()[3] 
        df_new['Age'][i]= temp
    elif (df_new['Age'][i].startswith("Age") and len(df_new['Age'][i])==13) :
        temp = df_new['Age'][i].split()[1][0:3]  
        df_new['Age'][i] = temp
    elif (df_new['Age'][i].startswith("Age") and len(df_new['Age'][i])==19) :   
        temp = df_new['Age'][i].split()[1]+"-5"
        df_new['Age'][i]= temp

df_new['Age'].unique()

array(['5-7', '4-7', '2-5', '0-12', '5-8', '3-8', '3-6', '8-12', '6-8',
       '3-5', '7-12', ' ', '0-4', '0-3', '1-3', '4-8', '2-6', '10-12',
       '12-14', '10-14', '3-7', '0-5', '6-12', '6-9', '7-9', '6-10',
       '0-9', '6-11', '8-11', '5-10', '2-4', '1-4', '0-2', '1-6', '7-10',
       '9-12', '4-6', '0-7', '6-7', '5-6', '7-8', '8-9', '9-10', '10-11',
       '11-12', '5-9', '1-5', '8-10', '0-6', '11-14', '5-12', '4-5',
       '3-12', '3-11', '2-12', '4-9', '4-10', '3-9', '7-11', '2-7',
       '0-11', '2-8', '3', '5', 'ages 3-5', '1-12', '9-11', '1-8', '4-11',
       ''], dtype=object)

In [16]:
#Data (length = 8) Eg. 'ages 3-5'
for i in df_new.index: 
    if(df_new['Age'][i].startswith("ages") and len(df_new['Age'][i])==8):
          temp = df_new['Age'][i].split()[1]
          df_new['Age'][i]= temp

In [17]:
# Updating the following ids based on 'Description' column 
# id - age range
# 1032 - 5-10     115 -  9-12     179 - 3-8     540 - 18+     586 - 3-8 
# 1090 - 0-5      339 - 3-5       235 - 5-7     805 - 15-18   1067 - 2-5 
# 440 - 2-5       782 - 2-5       285 - 2-5     77 -  8-12    1062 - 8-12   880 - 8-12


df_new['Age'][1032] = "5-10"
df_new['Age'][115] = "9-12"
df_new['Age'][179] = "3-8"
df_new['Age'][540] = "18+"
df_new['Age'][586] = "3-8"
df_new['Age'][1090] = "0-5"
df_new['Age'][339] = "3-5"
df_new['Age'][235] = "5-7"
df_new['Age'][805] = "15-18"
df_new['Age'][1067] = "2-5"
df_new['Age'][440] = "2-5"
df_new['Age'][782] = "2-5"
df_new['Age'][285] = "2-5"
df_new['Age'][77] = "8-12"
df_new['Age'][1062] = "8-12"
df_new['Age'][880] = "8-12"

In [18]:
#dropping rows with 'Age' column empty or blank

delrows = []
for i in df_new.index: 
    if (df_new['Age'][i]=="" or df_new['Age'][i]==" "):
        delrows.append(i)

df_new = df_new.drop(delrows)

len(df_new)


1148

In [19]:
#Joining df_3 and df_new 
# 1. Rename the column names of df_new according to df_3
df_new.rename(columns={'Age':'Reading_age', 'Description':'Desc', 'Name':'Title'}, inplace = True)

In [20]:
# 2. Join the dtaframes df_new to df_3.
books_df = pd.concat([df_4, df_new]).reset_index(drop=True)

books_df

,Title,Desc,Reading_age
0,A TRICK TRIKE,Little Ross was sad. He didnt have a nice new ...,2-9
1,ABE THE SERVICE DOG,Abe was a real Service Dog who dedicated his l...,2-9
2,BUBBLE FUN,"When the last bubble popped, he was gone but a...",2-9
3,CAPTAIN FANTASTIC,"Dr. Zobs evil plans had failed once again, th...",2-9
4,DO YOU WONDER WHY BUGS ARE GOOD?,Those little beauties have inspired so many st...,2-9
...,...,...,...
1577,You're God's Girl!: A Devotional for Tweens,\nDiscover the True You! Who is the real you? ...,8-11
1578,You're My Little Cuddle Bug,\nCelebrate your little cuddle bug with this c...,0-2
1579,Your Baby's First Word Will Be DADA,"\nYour baby's first word will be . . .""Dada!""R...",1-3
1580,Your Diary - Sparkly Lock & Keys - Girls 8+ Jo...,"\nBy You, about You, for You! Your Diary is al...",8-12


In [21]:
#The age range is of the form a-b
#make two columns in books_df as lowerAge(a) and upperAge(b)
#Reading_age are of the form - eg. '4-7', '9-10', '12-14', '18+' 
#The records with age range "18+"are taken as lower_age= 18 & upper_age=+
lowerAge=[]
upperAge=[]
l_age = 0
u_age = 0
age=0
for i in books_df.index: 
      if (books_df['Reading_age'][i].find('-')!=-1):
          age= books_df['Reading_age'][i].replace("-", " ")
          l_age= int(age.split()[0])
          u_age= int(age.split()[1])
      else:
          age= books_df['Reading_age'][i].replace("+", " ") 
          l_age= int(age.split()[0])
          u_age= 99
      lowerAge.append(l_age)
      upperAge.append(u_age)     

#syntax for inserting a new column
#DataFrameName.insert(loc, column, value, allow_duplicates = False)
books_df.insert(3, "lower_age", lowerAge, True)  
books_df.insert(4, "upper_age", upperAge, True)

books_df


,Title,Desc,Reading_age,lower_age,upper_age
0,A TRICK TRIKE,Little Ross was sad. He didnt have a nice new ...,2-9,2,9
1,ABE THE SERVICE DOG,Abe was a real Service Dog who dedicated his l...,2-9,2,9
2,BUBBLE FUN,"When the last bubble popped, he was gone but a...",2-9,2,9
3,CAPTAIN FANTASTIC,"Dr. Zobs evil plans had failed once again, th...",2-9,2,9
4,DO YOU WONDER WHY BUGS ARE GOOD?,Those little beauties have inspired so many st...,2-9,2,9
...,...,...,...,...,...
1577,You're God's Girl!: A Devotional for Tweens,\nDiscover the True You! Who is the real you? ...,8-11,8,11
1578,You're My Little Cuddle Bug,\nCelebrate your little cuddle bug with this c...,0-2,0,2
1579,Your Baby's First Word Will Be DADA,"\nYour baby's first word will be . . .""Dada!""R...",1-3,1,3
1580,Your Diary - Sparkly Lock & Keys - Girls 8+ Jo...,"\nBy You, about You, for You! Your Diary is al...",8-12,8,12


In [22]:
# deleting the record with upper-age as 99, since it is the only record in the Dataset. 
del99 = 0
for i in books_df.index: 
    if books_df['upper_age'][i] == 99 :
       del99 = i
      # print(del99)

books_df = books_df.drop(del99)

books_df

,Title,Desc,Reading_age,lower_age,upper_age
0,A TRICK TRIKE,Little Ross was sad. He didnt have a nice new ...,2-9,2,9
1,ABE THE SERVICE DOG,Abe was a real Service Dog who dedicated his l...,2-9,2,9
2,BUBBLE FUN,"When the last bubble popped, he was gone but a...",2-9,2,9
3,CAPTAIN FANTASTIC,"Dr. Zobs evil plans had failed once again, th...",2-9,2,9
4,DO YOU WONDER WHY BUGS ARE GOOD?,Those little beauties have inspired so many st...,2-9,2,9
...,...,...,...,...,...
1577,You're God's Girl!: A Devotional for Tweens,\nDiscover the True You! Who is the real you? ...,8-11,8,11
1578,You're My Little Cuddle Bug,\nCelebrate your little cuddle bug with this c...,0-2,0,2
1579,Your Baby's First Word Will Be DADA,"\nYour baby's first word will be . . .""Dada!""R...",1-3,1,3
1580,Your Diary - Sparkly Lock & Keys - Girls 8+ Jo...,"\nBy You, about You, for You! Your Diary is al...",8-12,8,12


The following records have lower and upper ages as 0-0 while it should be 8-12 	
Unnamed:0	Title		          Reading_age	lower_age	upper_age

36	36	A Wrinkle in Time		  0-0	         0	      0

411	411	Tom's Midnight Garden	0-0	         0	      0

In [23]:
books_df['lower_age'][36]=8
books_df['upper_age'][36]=12
books_df['Reading_age'][36]='8-12'

books_df['lower_age'][411]=8
books_df['upper_age'][411]=12
books_df['Reading_age'][411]='8-12'



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid 

In [24]:
## Records with single instances in data - 
##1-12, 1-2, 1-8, 11-12, 11-15, 15-18, 2-8, 3-11, 3-4, 7-8, 9-14

for i in books_df.index:
    if books_df['Reading_age'][i]=="2-8":
       print(books_df['Title'][i], str(i)) 

My Best Toddler Coloring Book - Fun with Numbers, Letters, Shapes, Colors, Animals: Big Activity Workbook for Toddlers & Kids 1029


In [25]:
books_df['Desc'][1190] = books_df['Title'][1190]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0-5 
6-10
11-14 
14-18 

if lower age <= 4, then set lower-age = 0, upper age = 5

else if lower age <= 9, then set lower-age = 6, upper age = 10 

else if lower age <= 13, then set lower-age = 11, upper age = 14

else if lower age <= 17, then set lower age = 14, upper age = 18

In [26]:

for i in books_df.index:
    l = books_df['lower_age'][i]
    u = books_df['upper_age'][i]
    if l<=4 :
       books_df['lower_age'][i] = 0
       books_df['upper_age'][i] = 5
    elif l<=9 :
       books_df['lower_age'][i] = 6
       books_df['upper_age'][i] = 10
    elif l<=17 :
       books_df['lower_age'][i] = 11
       books_df['upper_age'][i] = 18
 #   elif l<=17 :
 #      books_df['lower_age'][i] = 14
 #      books_df['upper_age'][i] = 18

    books_df['Reading_age'][i] = str(books_df['lower_age'][i])+"-"+str(books_df['upper_age'][i])        

books_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value i

,Title,Desc,Reading_age,lower_age,upper_age
0,A TRICK TRIKE,Little Ross was sad. He didnt have a nice new ...,0-5,0,5
1,ABE THE SERVICE DOG,Abe was a real Service Dog who dedicated his l...,0-5,0,5
2,BUBBLE FUN,"When the last bubble popped, he was gone but a...",0-5,0,5
3,CAPTAIN FANTASTIC,"Dr. Zobs evil plans had failed once again, th...",0-5,0,5
4,DO YOU WONDER WHY BUGS ARE GOOD?,Those little beauties have inspired so many st...,0-5,0,5
...,...,...,...,...,...
1577,You're God's Girl!: A Devotional for Tweens,\nDiscover the True You! Who is the real you? ...,6-10,6,10
1578,You're My Little Cuddle Bug,\nCelebrate your little cuddle bug with this c...,0-5,0,5
1579,Your Baby's First Word Will Be DADA,"\nYour baby's first word will be . . .""Dada!""R...",0-5,0,5
1580,Your Diary - Sparkly Lock & Keys - Girls 8+ Jo...,"\nBy You, about You, for You! Your Diary is al...",6-10,6,10


In [27]:
#Writing the records from books_df into csv file 'bookSynopsis_ageRange_clean'
#syntax: df.to_csv('file_name.csv')

books_df.to_csv('/gdrive/My Drive/MSC_Thesis_Project/Data/bookSynopsis_ageRange_clean.csv')
